con la tabla poliza unicamente puedo armar la matriz de cantidad de polizas por cliente en los meses

In [17]:
import pandas as pd
import numpy as np

In [32]:
df_poliza = pd.read_table("../../data/raw/TB_POLIZAS_CL.tsv", encoding="latin-1")
df_poliza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070858 entries, 0 to 4070857
Data columns (total 20 columns):
PROCESS_DATE            object
CUSTOMER_ID             int64
POLICY_ORIGINAL         float64
POLICY_NUMBER           int64
DATE_ISSUE              object
DATE_COMMENCENT_RISK    object
SUM_ASSURED             float64
PREMIUM                 object
PAYMENT_FREQUENCY       object
COD_VEH_MAKE            int64
VEH_MAKE                object
VEH_MODEL               object
VEH_YEAR                int64
CHANEL_ID               int64
CHANNEL                 object
NAME_OF_PRODUCT         object
TYPE_PRODUCT            object
COVERAGE_DETAILS        object
AGENT_ID                float64
SUB_AGENT_ID            float64
dtypes: float64(4), int64(5), object(11)
memory usage: 621.2+ MB


In [35]:
#convierto el due_date en fecha, para luego obtener el siguiente mes (correspondiente al servicio)
from datetime import datetime, timedelta
df_poliza["fecha"] = df_poliza.apply(lambda x : datetime.strptime(x["PROCESS_DATE"], "%d/%m/%Y"), axis=1)
df_poliza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070858 entries, 0 to 4070857
Data columns (total 21 columns):
PROCESS_DATE            object
CUSTOMER_ID             int64
POLICY_ORIGINAL         float64
POLICY_NUMBER           int64
DATE_ISSUE              object
DATE_COMMENCENT_RISK    object
SUM_ASSURED             float64
PREMIUM                 object
PAYMENT_FREQUENCY       object
COD_VEH_MAKE            int64
VEH_MAKE                object
VEH_MODEL               object
VEH_YEAR                int64
CHANEL_ID               int64
CHANNEL                 object
NAME_OF_PRODUCT         object
TYPE_PRODUCT            object
COVERAGE_DETAILS        object
AGENT_ID                float64
SUB_AGENT_ID            float64
fecha                   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(5), object(11)
memory usage: 652.2+ MB


In [36]:
df_poliza["next_f"] = df_poliza.fecha.apply(lambda x : x if pd.isnull(x) else datetime(x.year + (x.month // 12), ((x.month % 12) + 1), 1) )
df_poliza["mes"] = df_poliza.next_f.apply(lambda x : x if pd.isnull(x) else int(x.month) )
df_poliza["anio"] = df_poliza.next_f.apply(lambda x : x if pd.isnull(x) else int(x.year) )
df_poliza["corresp_date"] = df_poliza.next_f.apply(lambda x : x if pd.isnull(x) else "{}-{:02}".format(x.year, x.month) )
df_poliza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4070858 entries, 0 to 4070857
Data columns (total 25 columns):
PROCESS_DATE            object
CUSTOMER_ID             int64
POLICY_ORIGINAL         float64
POLICY_NUMBER           int64
DATE_ISSUE              object
DATE_COMMENCENT_RISK    object
SUM_ASSURED             float64
PREMIUM                 object
PAYMENT_FREQUENCY       object
COD_VEH_MAKE            int64
VEH_MAKE                object
VEH_MODEL               object
VEH_YEAR                int64
CHANEL_ID               int64
CHANNEL                 object
NAME_OF_PRODUCT         object
TYPE_PRODUCT            object
COVERAGE_DETAILS        object
AGENT_ID                float64
SUB_AGENT_ID            float64
fecha                   datetime64[ns]
next_f                  datetime64[ns]
mes                     int64
anio                    int64
corresp_date            object
dtypes: datetime64[ns](2), float64(4), int64(7), object(12)
memory usage: 776.5+ MB


In [19]:
#Uso el mes siguiente (correspondiente a la vigencia de la poliza)
#df_poliza["PROCESS_DATE_dia"]  = df_poliza.PROCESS_DATE.apply(lambda x : int(x.split("/")[0]))
#df_poliza["PROCESS_DATE_mes"]  = df_poliza.PROCESS_DATE.apply(lambda x : int(x.split("/")[1]))
#df_poliza["PROCESS_DATE_anio"] = df_poliza.PROCESS_DATE.apply(lambda x : int(x.split("/")[2]))
#df_poliza["mes_anio"] = df_poliza.PROCESS_DATE.apply(lambda x : x.split("/")[2]+"-"+x.split("/")[1])

In [37]:
df_poliza.sort_values(["anio", "mes"], inplace=True)

In [38]:
columns = df_poliza.corresp_date.unique()
row = df_poliza.CUSTOMER_ID.unique()

In [39]:
#creo el dataset vacio
bd = pd.DataFrame(columns=columns)
bd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Columns: 167 entries, 2004-03 to 2018-01
dtypes: object(167)
memory usage: 0.0+ bytes


In [40]:
#agrupo por cliente y mes_anio y cuento cuantas polizas distintas tiene
groups = df_poliza.groupby(["CUSTOMER_ID", "corresp_date"])["POLICY_NUMBER"].nunique()

In [41]:
for id_cliente in groups.index.levels[0]:
    bd.loc[id_cliente] = groups[id_cliente].T

In [42]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 1690 to 21260067
Columns: 167 entries, 2004-03 to 2018-01
dtypes: float64(167)
memory usage: 64.1 MB


In [43]:
bd.head()

,2004-03,2004-04,2004-05,2004-06,2004-07,2004-08,2004-09,2004-10,2004-11,2004-12,...,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01
1690,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0
7968,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [44]:
bd.to_csv("../../data/processed/BD_ID_CLIENTE_N.POLIZA_X_MES.csv")

In [31]:
df_poliza[df_poliza.CUSTOMER_ID == 9860].tail()

,PROCESS_DATE,CUSTOMER_ID,POLICY_ORIGINAL,POLICY_NUMBER,DATE_ISSUE,DATE_COMMENCENT_RISK,SUM_ASSURED,PREMIUM,PAYMENT_FREQUENCY,COD_VEH_MAKE,...,CHANNEL,NAME_OF_PRODUCT,TYPE_PRODUCT,COVERAGE_DETAILS,AGENT_ID,SUB_AGENT_ID,PROCESS_DATE_dia,PROCESS_DATE_mes,PROCESS_DATE_anio,mes_anio
734570,30/04/2017,9860,6.330010e+12,1649918119999,25/02/2017,29/12/2015,158700.0,"1642,28",MENSUAL,2015,...,CajaExpress,AUTO,TODO RIESGO C/ FRANQ. 5% C/RECUPERADOR -AG PA...,NaN,NaN,NaN,30,4,2017,2017-04
734571,31/05/2017,9860,6.330010e+12,1649918129999,25/02/2017,29/12/2015,158700.0,"1642,28",MENSUAL,2015,...,CajaExpress,AUTO,TODO RIESGO C/ FRANQ. 5% C/RECUPERADOR -AG PA...,NaN,NaN,NaN,31,5,2017,2017-05
3199057,30/06/2017,9860,6.330010e+12,1682129839999,21/05/2017,29/12/2015,158700.0,"1905,10",MENSUAL,2015,...,CajaExpress,AUTO,TODO RIESGO C/ FRANQ. 5% C/RECUPERADOR -AG PA...,NaN,NaN,NaN,30,6,2017,2017-06
734572,31/07/2017,9860,6.330010e+12,1682129859999,21/05/2017,29/12/2015,158700.0,"1905,10",MENSUAL,2015,...,CajaExpress,AUTO,TODO RIESGO C/ FRANQ. 5% C/RECUPERADOR -AG PA...,NaN,NaN,NaN,31,7,2017,2017-07
734573,31/08/2017,9860,6.330010e+12,1682129869999,21/05/2017,29/12/2015,158700.0,"1905,10",MENSUAL,2015,...,CajaExpress,AUTO,TODO RIESGO C/ FRANQ. 5% C/RECUPERADOR -AG PA...,NaN,NaN,NaN,31,8,2017,2017-08
